In [1]:
# Import package
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from pytorchtools import EarlyStopping

from sklearn.metrics import mean_squared_error,r2_score

# show plot directly
%matplotlib inline

In [2]:
# load .csv data
data_path = './Data/Example_original_5km.csv'
meta = pd.read_csv(data_path)
meta.head()

,Unnamed: 0,FID,year,month,doy,site,type,lat,lon,MCD43_B1,...,V80,V60,V40,V20,L80,L60,L40,L20,SW_GLASS,GPP_VUT_NT_MEAN
0,356,1,2009,12,356,AR-SLu,MF,-33.4648,-66.4598,0.0821,...,0.276458,0.240926,0.205394,0.169861,336.3,308.2,204.2,123.6,271.85,9.39258
1,358,1,2009,12,358,AR-SLu,MF,-33.4648,-66.4598,0.0815,...,0.276458,0.240926,0.205394,0.169861,336.3,308.2,204.2,123.6,304.58,9.09283
2,359,1,2009,12,359,AR-SLu,MF,-33.4648,-66.4598,0.0779,...,0.276458,0.240926,0.205394,0.169861,336.3,308.2,204.2,123.6,250.98,7.43662
3,360,1,2009,12,360,AR-SLu,MF,-33.4648,-66.4598,0.0798,...,0.276458,0.240926,0.205394,0.169861,336.3,308.2,204.2,123.6,317.05,10.46630
4,361,1,2009,12,361,AR-SLu,MF,-33.4648,-66.4598,0.0798,...,0.276458,0.240926,0.205394,0.169861,336.3,308.2,204.2,123.6,282.45,9.27425


In [3]:
# Extract the numeric variables
base = meta.iloc[:,1:9]

data_up = meta.iloc[:,9:]
data_up.head()

,MCD43_B1,MCD43_B2,MCD43_B3,MCD43_B4,MCD43_B6,Top,Bottom,V80,V60,V40,V20,L80,L60,L40,L20,SW_GLASS,GPP_VUT_NT_MEAN
0,0.0821,0.2395,0.0439,0.0768,0.2501,0.31199,0.134329,0.276458,0.240926,0.205394,0.169861,336.3,308.2,204.2,123.6,271.85,9.39258
1,0.0815,0.2438,0.0437,0.0767,0.2529,0.31199,0.134329,0.276458,0.240926,0.205394,0.169861,336.3,308.2,204.2,123.6,304.58,9.09283
2,0.0779,0.2371,0.0416,0.0740,0.2440,0.31199,0.134329,0.276458,0.240926,0.205394,0.169861,336.3,308.2,204.2,123.6,250.98,7.43662
3,0.0798,0.2384,0.0429,0.0755,0.2491,0.31199,0.134329,0.276458,0.240926,0.205394,0.169861,336.3,308.2,204.2,123.6,317.05,10.46630
4,0.0798,0.2386,0.0429,0.0754,0.2491,0.31199,0.134329,0.276458,0.240926,0.205394,0.169861,336.3,308.2,204.2,123.6,282.45,9.27425


In [4]:
# Pre process (normalization)
quant_features = data_up.columns
print(quant_features)

scaled_features = {}
for each in quant_features:
    mean, std = data_up[each].mean(), data_up[each].std()
    scaled_features[each] = [mean, std]
    data_up.loc[:, each] = (data_up[each] - mean)/std

Index(['MCD43_B1', 'MCD43_B2', 'MCD43_B3', 'MCD43_B4', 'MCD43_B6', 'Top',
       'Bottom', 'V80', 'V60', 'V40', 'V20', 'L80', 'L60', 'L40', 'L20',
       'SW_GLASS', 'GPP_VUT_NT_MEAN'],
      dtype='object')


In [5]:
features = data_up.drop(['GPP_VUT_NT_MEAN'], axis =1)
features.head()

,MCD43_B1,MCD43_B2,MCD43_B3,MCD43_B4,MCD43_B6,Top,Bottom,V80,V60,V40,V20,L80,L60,L40,L20,SW_GLASS
0,0.112901,-0.221532,-0.044448,0.052619,0.709414,-1.26663,-0.540437,-1.329437,-1.378643,-1.328026,-1.02253,2.317797,1.422392,-0.207623,-1.203221,0.945649
1,0.103753,-0.165686,-0.047716,0.050929,0.741686,-1.26663,-0.540437,-1.329437,-1.378643,-1.328026,-1.02253,2.317797,1.422392,-0.207623,-1.203221,1.276692
2,0.048865,-0.252702,-0.082030,0.005293,0.639107,-1.26663,-0.540437,-1.329437,-1.378643,-1.328026,-1.02253,2.317797,1.422392,-0.207623,-1.203221,0.734562
3,0.077834,-0.235818,-0.060788,0.030646,0.697888,-1.26663,-0.540437,-1.329437,-1.378643,-1.328026,-1.02253,2.317797,1.422392,-0.207623,-1.203221,1.402818
4,0.077834,-0.233221,-0.060788,0.028956,0.697888,-1.26663,-0.540437,-1.329437,-1.378643,-1.328026,-1.02253,2.317797,1.422392,-0.207623,-1.203221,1.052861


In [6]:
# define MLP 
input_size = features.shape[1]
hidden_size = [18,9,3]
output_size = 1

class GPPNet(nn.Module):
    def __init__(self):
        super(GPPNet, self).__init__()
        
        self.layer1 = nn.Linear(input_size, hidden_size[0])
        self.layer2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.layer3 = nn.Linear(hidden_size[1], hidden_size[2])
        self.out = nn.Linear(hidden_size[2], output_size)

    def forward(self, x):
        x = torch.sigmoid(self.layer1(x))
        x = torch.sigmoid(self.layer2(x))
        x = torch.sigmoid(self.layer3(x))
        x = self.out(x)
        return x
    
neu = GPPNet()
cost = torch.nn.MSELoss()
#optimizer = torch.optim.SGD(neu.parameters(), lr = 0.01)

In [7]:
result = base
# estimate GPP with 10-fold submodel
for i in range(10):
    n = i+1
    neu = torch.load('ETERS_F0' + str(n) + '.mdl')
    x = torch.tensor(features.values, dtype = torch.float, requires_grad = True)
    neu = neu.cpu().eval()
    predict = neu(x)
    predict = predict.data.numpy()
    
    mean, std = scaled_features['GPP_VUT_NT_MEAN']
    est = predict * std + mean

    estimation = pd.DataFrame(est) # GPP_B_G_mGreen
    estimation.columns = ['GPP_25pixel_Near_Fold'+str(n)]
    estimation.head()
    
    result = pd.concat([result, estimation],axis = 1)

In [8]:
result.head()
#result.to_csv(r'./Estimation_result.csv', index=False)

,FID,year,month,doy,site,type,lat,lon,GPP_25pixel_Near_Fold1,GPP_25pixel_Near_Fold2,GPP_25pixel_Near_Fold3,GPP_25pixel_Near_Fold4,GPP_25pixel_Near_Fold5,GPP_25pixel_Near_Fold6,GPP_25pixel_Near_Fold7,GPP_25pixel_Near_Fold8,GPP_25pixel_Near_Fold9,GPP_25pixel_Near_Fold10
0,1,2009,12,356,AR-SLu,MF,-33.4648,-66.4598,4.381586,4.547682,5.049779,7.087522,6.106876,7.471730,7.789704,4.570056,8.540238,4.383664
1,1,2009,12,358,AR-SLu,MF,-33.4648,-66.4598,4.600446,4.791988,6.082608,7.228041,6.580718,8.387282,7.820419,5.647984,8.888798,4.533598
2,1,2009,12,359,AR-SLu,MF,-33.4648,-66.4598,4.211583,4.528721,5.580623,6.943994,6.091410,7.343781,7.549306,4.539117,8.365333,4.505333
3,1,2009,12,360,AR-SLu,MF,-33.4648,-66.4598,4.449154,4.793029,6.041151,7.164734,6.397151,9.434757,7.814329,5.755575,8.852182,4.588972
4,1,2009,12,361,AR-SLu,MF,-33.4648,-66.4598,4.368369,4.648773,5.664474,7.120511,6.274310,8.073658,7.707404,4.966887,8.650322,4.488078
